# p2016a_input_F.ipynb
## Generate equilibrium (F) MAM3 input data (emissions and user_nl_cam files) for p2016a_hist_reg

## History:
- 2016-07 - Benjamin S. Grandey (benjamin@smart.mit.edu).
- 2016-07-29 - BSG. Finishing, finalizing and re-running in preparation for submitting simulations.

## Requirements:
- template_user_nl_cam_p16a_F - template for user_nl_cam namelist files.
- Standard CESM MAM3 emissions data files.
- Python modules mentioned in next cell.

In [1]:
import datetime
import netCDF4
import numpy as np
import os
import shutil

%matplotlib inline

!date

Fri Jul 29 16:15:52 SGT 2016


In [2]:
# Directories where data lives
mam3_dir = '/users/grandey/data/inputdataCESM/trop_mozart_aero/emis/'  # Location of standard CESM MAM3 emissions data
data_dir = '/users/grandey/data/projects/p2016a_hist_reg/'  # Root directory for project
input_dir = data_dir + 'input/p16a_F/'  # Directory in which to write the data produced by this notebook

# Define regions
region_dict = {'EAs'  : [94, 156, 20, 65],
               'SEAs' : [94, 161, -10, 20],
               'ANZ'  : [109, 179, -50, -10],
               'SAs'  : [61, 94, 0, 35],
               'AfME' : [-21, 61, -40, 35],
               'Eur'  : [-26, 31, 35, 75],
               'CAs'  : [31, 94, 35, 75],
               'NAm'  : [-169, -51, 15, 75],
               'SAm'  : [-94, -31, -60, 15],}
# Longitudes have been deliberately chosen not to coincide with longitudes of grid

# Species-level combinations that need modifying
species_level_list = ['so2_surf', 'bc_surf', 'oc_surf', 'so4_a1_surf', 'so4_a2_surf', 'num_a1_surf', 'num_a2_surf',
                      'so2_elev', 'so4_a1_elev', 'num_a1_elev',
                      'bc_elev', 'oc_elev', 'so4_a2_elev', 'num_a2_elev']
# The final four species contain only biomass and volcanic emissions, so No* and Only* scenarios are same as Hist.

!date

Fri Jul 29 16:15:52 SGT 2016


## p16a\_F\_* equilibrium single-year emissions files for prescribed-SST simulations

In [3]:
# List of scenarios for which emissions files have been produced - initially empty
scenario_list = []

In [4]:
# p16a_F_Hist_*: Equilibrium emissions files for scenario "Hist", for 1970 and 2000
# Loop over years of interest and species-level combinations
for emissions_year in (1970, 2000):
    for species_level in species_level_list:
        # Delete temporary and output files
        temp_filename = input_dir+'temp_p16a_F_Hist_'+str(emissions_year)+'_mam3_'+species_level+'.nc'
        out_filename = input_dir+'p16a_F_Hist_'+str(emissions_year)+'_mam3_'+species_level+'.nc'
        for filename in [temp_filename, out_filename]:
            if os.path.isfile(filename):
                os.remove(filename)
        # Copy standard ar5_mam3 year-2000 file as a template
        template_filename = mam3_dir+'ar5_mam3_'+species_level+'_2000_c090726.nc'
        shutil.copy(template_filename, temp_filename)
        !chmod +w {temp_filename}
        # Open temp file for editing
        temp_nc = netCDF4.Dataset(temp_filename, 'a')
        # Open ar5_mam3 1850-2000 file for copying data
        in_filename = mam3_dir+'ar5_mam3_'+species_level+'_1850-2005_c090804.nc'
        in_nc = netCDF4.Dataset(in_filename, 'r')
        # Find time locations of emissions_year using date dimension of in_nc
        in_date_t = in_nc['date'][:]
        t_index_array = np.where((in_date_t/1e4).astype('int') == emissions_year)[0]
        #print "in_date_t[t_index_array] =", in_date_t[t_index_array]
        # Loop over variables and replace all emissions (including natural) with those corresponding to emissions year
        for var_name in temp_nc.variables:
            if var_name not in ['date', 'lat', 'lon', 'altitude', 'altitude_int']:
                if species_level.split('_')[-1] == 'surf':
                    temp_nc[var_name][:, :, :] = in_nc[var_name][t_index_array, :, :]
                else:
                    temp_nc[var_name][:, :, :, :] = in_nc[var_name][t_index_array, :, :, :]
        # History attribute
        temp_nc.history = (datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')+
                           ': p2016a_input_F.ipynb: created '+out_filename.split('/')[-1]+
                           ' using data from '+in_filename.split('/')[-1]+', and '+
                           template_filename.split('/')[-1]+' as a template.')
        temp_nc.created_by = 'Benjamin S. Grandey, using p2016a_input_F.ipynb and ar5_mam3_* emissions files.'
        # Close NetCDF files
        in_nc.close()
        temp_nc.close()
        # Move temp_filename to out_filename
        shutil.move(temp_filename, out_filename)
        print "Written", out_filename
    # Append scenario to list
    scenario_list.append('Hist_'+str(emissions_year))
        
!date

Written /users/grandey/data/projects/p2016a_hist_reg/input/p16a_F/p16a_F_Hist_1970_mam3_so2_surf.nc
Written /users/grandey/data/projects/p2016a_hist_reg/input/p16a_F/p16a_F_Hist_1970_mam3_bc_surf.nc
Written /users/grandey/data/projects/p2016a_hist_reg/input/p16a_F/p16a_F_Hist_1970_mam3_oc_surf.nc
Written /users/grandey/data/projects/p2016a_hist_reg/input/p16a_F/p16a_F_Hist_1970_mam3_so4_a1_surf.nc
Written /users/grandey/data/projects/p2016a_hist_reg/input/p16a_F/p16a_F_Hist_1970_mam3_so4_a2_surf.nc
Written /users/grandey/data/projects/p2016a_hist_reg/input/p16a_F/p16a_F_Hist_1970_mam3_num_a1_surf.nc
Written /users/grandey/data/projects/p2016a_hist_reg/input/p16a_F/p16a_F_Hist_1970_mam3_num_a2_surf.nc
Written /users/grandey/data/projects/p2016a_hist_reg/input/p16a_F/p16a_F_Hist_1970_mam3_so2_elev.nc
Written /users/grandey/data/projects/p2016a_hist_reg/input/p16a_F/p16a_F_Hist_1970_mam3_so4_a1_elev.nc
Written /users/grandey/data/projects/p2016a_hist_reg/input/p16a_F/p16a_F_Hist_1970_mam3

In [5]:
# p16a_F_Zero_*: Equilibrium emissions files for scenario "Zero", with no anth land or ship, for 1970 and 2000
# Loop over years of interest and species-level combinations
for emissions_year in (1970, 2000):
    for species_level in species_level_list:
        # Delete temporary and output files
        temp_filename = input_dir+'temp_p16a_F_Zero_'+str(emissions_year)+'_mam3_'+species_level+'.nc'
        out_filename = input_dir+'p16a_F_Zero_'+str(emissions_year)+'_mam3_'+species_level+'.nc'
        for filename in [temp_filename, out_filename]:
            if os.path.isfile(filename):
                os.remove(filename)
        # Copy p16a_F_Hist_* file as base
        in_filename = input_dir+'p16a_F_Hist_'+str(emissions_year)+'_mam3_'+species_level+'.nc'
        shutil.copy(in_filename, temp_filename)
        # Open temp file for editing
        temp_nc = netCDF4.Dataset(temp_filename, 'a')
        # Loop over emissions categories
        for var_name in temp_nc.variables:
            # Set anthropogenic land and ship emissions to zero
            if 'emiss' in var_name:
                temp_nc[var_name][:] = 0
        # History attribute
        temp_nc.history = (datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')+
                           ': p2016a_input_F.ipynb: created '+out_filename.split('/')[-1]+
                           ' using data from '+in_filename.split('/')[-1]+'.')
        temp_nc.created_by = 'Benjamin S. Grandey, using p2016a_input_F.ipynb and ar5_mam3_* emissions files.'
        # Close temp file
        temp_nc.close()
        # Move temp_filename to out_filename
        shutil.move(temp_filename, out_filename)
        print "Written", out_filename
    # Append scenario to list
    scenario_list.append('Zero_'+str(emissions_year))

!date

Written /users/grandey/data/projects/p2016a_hist_reg/input/p16a_F/p16a_F_Zero_1970_mam3_so2_surf.nc
Written /users/grandey/data/projects/p2016a_hist_reg/input/p16a_F/p16a_F_Zero_1970_mam3_bc_surf.nc
Written /users/grandey/data/projects/p2016a_hist_reg/input/p16a_F/p16a_F_Zero_1970_mam3_oc_surf.nc
Written /users/grandey/data/projects/p2016a_hist_reg/input/p16a_F/p16a_F_Zero_1970_mam3_so4_a1_surf.nc
Written /users/grandey/data/projects/p2016a_hist_reg/input/p16a_F/p16a_F_Zero_1970_mam3_so4_a2_surf.nc
Written /users/grandey/data/projects/p2016a_hist_reg/input/p16a_F/p16a_F_Zero_1970_mam3_num_a1_surf.nc
Written /users/grandey/data/projects/p2016a_hist_reg/input/p16a_F/p16a_F_Zero_1970_mam3_num_a2_surf.nc
Written /users/grandey/data/projects/p2016a_hist_reg/input/p16a_F/p16a_F_Zero_1970_mam3_so2_elev.nc
Written /users/grandey/data/projects/p2016a_hist_reg/input/p16a_F/p16a_F_Zero_1970_mam3_so4_a1_elev.nc
Written /users/grandey/data/projects/p2016a_hist_reg/input/p16a_F/p16a_F_Zero_1970_mam3

In [6]:
# p16a_F_No* and p16a_F_Only* for regions: Equilibrium emissions files for "No" and "Only" scenarios, for 1970 and 2000
# Loop over scenario combinations ("No" vs "Only", and emissions year), regions, and species-level combinations
for no_or_only, emissions_year in [('No', 2000), ('No', 1970), ('Only', 2000)]:
    for reg_name, reg_bounds in region_dict.items():
        for species_level in species_level_list:
            # Delete temporary and final output files
            temp_filename = (input_dir+'temp_p16a_F_'+no_or_only+reg_name+'_'+str(emissions_year)+
                             '_mam3_'+species_level+'.nc')
            out_filename = (input_dir+'p16a_F_'+no_or_only+reg_name+'_'+str(emissions_year)+
                            '_mam3_'+species_level+'.nc')
            for filename in [temp_filename, out_filename]:
                if os.path.isfile(filename):
                    os.remove(filename)
            # Copy p16a_F_Hist_* file as base
            in_filename = input_dir+'p16a_F_Hist_'+str(emissions_year)+'_mam3_'+species_level+'.nc'
            shutil.copy(in_filename, temp_filename)
            # Open temp file for editing
            temp_nc = netCDF4.Dataset(temp_filename, 'a')
            # Get longitude and latitude dimensions
            lon_x = temp_nc['lon'][:]
            lat_y = temp_nc['lat'][:]
            # Indices of region bounds. Note: region_dict uses -180 to 180, while MAM3 files use 0 to 360
            if reg_bounds[0] >= 0:
                x_bot = np.where(lon_x >= reg_bounds[0])[0][0]
            else:
                x_bot = np.where(lon_x >= (reg_bounds[0]+360.))[0][0]
            if reg_bounds[1] >= 0: 
                x_top = np.where(lon_x <= reg_bounds[1])[0][-1]
            else:
                x_top = np.where(lon_x <= (reg_bounds[1]+360.))[0][-1]
            y_bot = np.where(lat_y >= reg_bounds[2])[0][0]
            y_top = np.where(lat_y <= reg_bounds[3])[0][-1]
            # Loop over anthropogenic land emissions categories
            for var_name in temp_nc.variables:
                if 'emiss' in var_name:
                    # Load data, and map to 4D for consistency
                    if species_level.split('_')[-1] == 'surf':
                        nt, ny, nx = temp_nc[var_name][:].shape
                        nz = 1
                        data_tzyx = np.reshape(temp_nc[var_name][:], [nt, 1, ny, nx])
                    else:
                        nt, nz, ny, nx = temp_nc[var_name][:].shape
                        data_tzyx = temp_nc[var_name][:]
                    # If 'No', set to zero within region
                    if no_or_only == 'No' and 'shp' not in var_name:
                        if x_top > x_bot:
                            data_tzyx[:, :, y_bot:y_top+1, x_bot:x_top+1] = 0
                        else:
                            data_tzyx[:, :, y_bot:y_top+1, x_bot:] = 0
                            data_tzyx[:, :, y_bot:y_top+1, :x_top+1] = 0
                    # If 'Only', set to zero outside region, and also set ship to zero
                    if no_or_only == 'Only':
                        if 'shp' in var_name:
                            data_tzyx[:] = 0
                        else:
                            if x_top > x_bot:
                                data_tzyx[:, :, :, :x_bot] = 0
                                data_tzyx[:, :, :, x_top+1:] = 0
                            else:
                                data_tzyx[:, :, :, x_top+1:x_bot] = 0
                            data_tzyx[:, :, :y_bot, :] = 0
                            data_tzyx[:, :, y_top+1:, :] = 0
                    # Write modified data to file
                    temp_nc[var_name][:] = data_tzyx[:]
            # History attribute
            temp_nc.history = (datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')+
                           ': p2016a_input_F.ipynb: created '+out_filename.split('/')[-1]+
                           ' using data from '+in_filename.split('/')[-1]+'.')
            temp_nc.created_by = 'Benjamin S. Grandey, using p2016a_input_F.ipynb and ar5_mam3_* emissions files.'
            # Close temp file
            temp_nc.close()
            # Move temp_filename to out_filename
            shutil.move(temp_filename, out_filename)
            print "Written", out_filename
        # Append scenario to list
        scenario_list.append(no_or_only+reg_name+'_'+str(emissions_year))
        
!date

Written /users/grandey/data/projects/p2016a_hist_reg/input/p16a_F/p16a_F_NoSAs_2000_mam3_so2_surf.nc
Written /users/grandey/data/projects/p2016a_hist_reg/input/p16a_F/p16a_F_NoSAs_2000_mam3_bc_surf.nc
Written /users/grandey/data/projects/p2016a_hist_reg/input/p16a_F/p16a_F_NoSAs_2000_mam3_oc_surf.nc
Written /users/grandey/data/projects/p2016a_hist_reg/input/p16a_F/p16a_F_NoSAs_2000_mam3_so4_a1_surf.nc
Written /users/grandey/data/projects/p2016a_hist_reg/input/p16a_F/p16a_F_NoSAs_2000_mam3_so4_a2_surf.nc
Written /users/grandey/data/projects/p2016a_hist_reg/input/p16a_F/p16a_F_NoSAs_2000_mam3_num_a1_surf.nc
Written /users/grandey/data/projects/p2016a_hist_reg/input/p16a_F/p16a_F_NoSAs_2000_mam3_num_a2_surf.nc
Written /users/grandey/data/projects/p2016a_hist_reg/input/p16a_F/p16a_F_NoSAs_2000_mam3_so2_elev.nc
Written /users/grandey/data/projects/p2016a_hist_reg/input/p16a_F/p16a_F_NoSAs_2000_mam3_so4_a1_elev.nc
Written /users/grandey/data/projects/p2016a_hist_reg/input/p16a_F/p16a_F_NoSAs

In [7]:
# p16a_F_NoShip and p16a_F_OnlyShip: Equilibrium emissions files for "NoShip" and "OnlyShip" scenarios, for 1970 and 2000
# Loop over scenario combinations ("No" vs "Only", and emissions year) and species-level combinations
for no_or_only, emissions_year in [('No', 2000), ('No', 1970), ('Only', 2000)]:
    for species_level in species_level_list:
        # Delete temporary and final output files
        temp_filename = (input_dir+'temp_p16a_F_'+no_or_only+'Ship_'+str(emissions_year)+
                         '_mam3_'+species_level+'.nc')
        out_filename = (input_dir+'p16a_F_'+no_or_only+'Ship_'+str(emissions_year)+
                        '_mam3_'+species_level+'.nc')
        for filename in [temp_filename, out_filename]:
            if os.path.isfile(filename):
                os.remove(filename)
        # Copy p16a_F_Hist_* file as base
        in_filename = input_dir+'p16a_F_Hist_'+str(emissions_year)+'_mam3_'+species_level+'.nc'
        shutil.copy(in_filename, temp_filename)
        # Open temp file for editing
        temp_nc = netCDF4.Dataset(temp_filename, 'a')
        # Loop over emissions categories
        for var_name in temp_nc.variables:
            if var_name not in ['date', 'lat', 'lon', 'altitude', 'altitude_int']:
                if no_or_only == 'No':
                    # If 'No', delete ship emissions
                    if 'shp' in var_name:
                        temp_nc[var_name][:] = 0
                else:
                    # If 'Only', delete all other anthropogenic emissions
                    if 'emiss' in var_name and 'shp' not in var_name:
                        temp_nc[var_name][:] = 0
        # History attribute
        temp_nc.history = (datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')+
                           ': p2016a_input_F.ipynb: created '+out_filename.split('/')[-1]+
                           ' using data from '+in_filename.split('/')[-1]+'.')
        temp_nc.created_by = 'Benjamin S. Grandey, using p2016a_input_F.ipynb and ar5_mam3_* emissions files.'
        # Close temp file
        temp_nc.close()
        # Move temp_filename to out_filename
        shutil.move(temp_filename, out_filename)
        print "Written", out_filename
    # Append scenario to list
    scenario_list.append(no_or_only+'Ship_'+str(emissions_year))
        
!date

Written /users/grandey/data/projects/p2016a_hist_reg/input/p16a_F/p16a_F_NoShip_2000_mam3_so2_surf.nc
Written /users/grandey/data/projects/p2016a_hist_reg/input/p16a_F/p16a_F_NoShip_2000_mam3_bc_surf.nc
Written /users/grandey/data/projects/p2016a_hist_reg/input/p16a_F/p16a_F_NoShip_2000_mam3_oc_surf.nc
Written /users/grandey/data/projects/p2016a_hist_reg/input/p16a_F/p16a_F_NoShip_2000_mam3_so4_a1_surf.nc
Written /users/grandey/data/projects/p2016a_hist_reg/input/p16a_F/p16a_F_NoShip_2000_mam3_so4_a2_surf.nc
Written /users/grandey/data/projects/p2016a_hist_reg/input/p16a_F/p16a_F_NoShip_2000_mam3_num_a1_surf.nc
Written /users/grandey/data/projects/p2016a_hist_reg/input/p16a_F/p16a_F_NoShip_2000_mam3_num_a2_surf.nc
Written /users/grandey/data/projects/p2016a_hist_reg/input/p16a_F/p16a_F_NoShip_2000_mam3_so2_elev.nc
Written /users/grandey/data/projects/p2016a_hist_reg/input/p16a_F/p16a_F_NoShip_2000_mam3_so4_a1_elev.nc
Written /users/grandey/data/projects/p2016a_hist_reg/input/p16a_F/p16

## user\_nl\_cam\_p16a\_F\_\* namelist files for prescribed SST runs

In [8]:
# Loop over scenarios
for scenario in scenario_list:
    # Open template file for reading
    in_filename = 'template_user_nl_cam_p16a_F'
    in_file = open(in_filename, 'r')
    # Open output file for writing, after deleting any previous version
    out_filename = input_dir+'user_nl_cam_p16a_F_'+scenario
    if os.path.isfile(out_filename):
        os.remove(out_filename)
    out_file = open(out_filename, 'w')
    # Loop over lines in template file
    for in_line in in_file:
        # Edit or copy line and write to output file
        out_line = scenario.join(in_line.split('<SCENARIO>'))
        out_file.write(out_line)
    # Close files
    in_file.close()
    out_file.close()
    print "Written", out_filename

!date

Written /users/grandey/data/projects/p2016a_hist_reg/input/p16a_F/user_nl_cam_p16a_F_Hist_1970
Written /users/grandey/data/projects/p2016a_hist_reg/input/p16a_F/user_nl_cam_p16a_F_Hist_2000
Written /users/grandey/data/projects/p2016a_hist_reg/input/p16a_F/user_nl_cam_p16a_F_Zero_1970
Written /users/grandey/data/projects/p2016a_hist_reg/input/p16a_F/user_nl_cam_p16a_F_Zero_2000
Written /users/grandey/data/projects/p2016a_hist_reg/input/p16a_F/user_nl_cam_p16a_F_NoSAs_2000
Written /users/grandey/data/projects/p2016a_hist_reg/input/p16a_F/user_nl_cam_p16a_F_NoCAs_2000
Written /users/grandey/data/projects/p2016a_hist_reg/input/p16a_F/user_nl_cam_p16a_F_NoSAm_2000
Written /users/grandey/data/projects/p2016a_hist_reg/input/p16a_F/user_nl_cam_p16a_F_NoAfME_2000
Written /users/grandey/data/projects/p2016a_hist_reg/input/p16a_F/user_nl_cam_p16a_F_NoSEAs_2000
Written /users/grandey/data/projects/p2016a_hist_reg/input/p16a_F/user_nl_cam_p16a_F_NoEAs_2000
Written /users/grandey/data/projects/p2016

# Tar the input data, ready for syncing to supercomputer

In [9]:
# Change directory to location where tarball will be created
original_wd = os.getcwd()
os.chdir(data_dir + 'input')
print "Now in", os.getcwd()
# Create tarball
tar_filename = 'input_p16a_F_' + datetime.datetime.now().strftime('%Y-%m-%d_%H-%M') + '.tar.gz'
if os.path.isfile(tar_filename):
    os.remove(tar_filename)
!tar --exclude='.*' -cvzf $tar_filename 'p16a_F'
print "Written", tar_filename
# Change directory to location where tarball will be created
os.chdir(original_wd)
print "Now in", os.getcwd()

!date

Now in /Users/grandey/data/projects/p2016a_hist_reg/input
a p16a_F
a p16a_F/p16a_F_Hist_1970_mam3_bc_elev.nc
a p16a_F/p16a_F_Hist_1970_mam3_bc_surf.nc
a p16a_F/p16a_F_Hist_1970_mam3_num_a1_elev.nc
a p16a_F/p16a_F_Hist_1970_mam3_num_a1_surf.nc
a p16a_F/p16a_F_Hist_1970_mam3_num_a2_elev.nc
a p16a_F/p16a_F_Hist_1970_mam3_num_a2_surf.nc
a p16a_F/p16a_F_Hist_1970_mam3_oc_elev.nc
a p16a_F/p16a_F_Hist_1970_mam3_oc_surf.nc
a p16a_F/p16a_F_Hist_1970_mam3_so2_elev.nc
a p16a_F/p16a_F_Hist_1970_mam3_so2_surf.nc
a p16a_F/p16a_F_Hist_1970_mam3_so4_a1_elev.nc
a p16a_F/p16a_F_Hist_1970_mam3_so4_a1_surf.nc
a p16a_F/p16a_F_Hist_1970_mam3_so4_a2_elev.nc
a p16a_F/p16a_F_Hist_1970_mam3_so4_a2_surf.nc
a p16a_F/p16a_F_Hist_2000_mam3_bc_elev.nc
a p16a_F/p16a_F_Hist_2000_mam3_bc_surf.nc
a p16a_F/p16a_F_Hist_2000_mam3_num_a1_elev.nc
a p16a_F/p16a_F_Hist_2000_mam3_num_a1_surf.nc
a p16a_F/p16a_F_Hist_2000_mam3_num_a2_elev.nc
a p16a_F/p16a_F_Hist_2000_mam3_num_a2_surf.nc
a p16a_F/p16a_F_Hist_2000_mam3_oc_elev.nc
a

## Configuration instructions for p16a_F simulations

In [10]:
# Convert list of scenarios to strings that can be copied and pasted for bash loops on supercomputer
print 'All scenarios:'
print ' '.join(scenario_list)
print
print 'Hist and Zero scenarios:'
print ' '.join(s for s in scenario_list if 'Hist' in s or 'Zero' in s)
print
print 'No-2000 scenarios:'
print ' '.join(s for s in scenario_list if s[0:2] == 'No' and s[-4:] == '2000')
print
print 'Only-2000 scenarios:'
print ' '.join(s for s in scenario_list if s[0:4] == 'Only' and s[-4:] == '2000')
print
print 'No-1970 scenarios:'
print ' '.join(s for s in scenario_list if s[0:2] == 'No' and s[-4:] == '1970')
print
!date

All scenarios:
Hist_1970 Hist_2000 Zero_1970 Zero_2000 NoSAs_2000 NoCAs_2000 NoSAm_2000 NoAfME_2000 NoSEAs_2000 NoEAs_2000 NoNAm_2000 NoANZ_2000 NoEur_2000 NoSAs_1970 NoCAs_1970 NoSAm_1970 NoAfME_1970 NoSEAs_1970 NoEAs_1970 NoNAm_1970 NoANZ_1970 NoEur_1970 OnlySAs_2000 OnlyCAs_2000 OnlySAm_2000 OnlyAfME_2000 OnlySEAs_2000 OnlyEAs_2000 OnlyNAm_2000 OnlyANZ_2000 OnlyEur_2000 NoShip_2000 NoShip_1970 OnlyShip_2000

Hist and Zero scenarios:
Hist_1970 Hist_2000 Zero_1970 Zero_2000

No-2000 scenarios:
NoSAs_2000 NoCAs_2000 NoSAm_2000 NoAfME_2000 NoSEAs_2000 NoEAs_2000 NoNAm_2000 NoANZ_2000 NoEur_2000 NoShip_2000

Only-2000 scenarios:
OnlySAs_2000 OnlyCAs_2000 OnlySAm_2000 OnlyAfME_2000 OnlySEAs_2000 OnlyEAs_2000 OnlyNAm_2000 OnlyANZ_2000 OnlyEur_2000 OnlyShip_2000

No-1970 scenarios:
NoSAs_1970 NoCAs_1970 NoSAm_1970 NoAfME_1970 NoSEAs_1970 NoEAs_1970 NoNAm_1970 NoANZ_1970 NoEur_1970 NoShip_1970

Fri Jul 29 16:17:47 SGT 2016


### Part A: on local machine

Use rsync to sync p16a\_F\_\* input data tarball to NSCC cluster ~/scratch/00Data4Modeling/inputdataBSG/.

### Part B: on supercomputer (NSCC cluster)

Unpack the p16a\_F\_\* input data in the inputdataBSG directory.

    cd ~/scratch/00Data4Modeling/inputdataBSG/
    tar -xvzf $tar_filename

Create environment variable containing list of scenarios (either all or batch).

    P16A_F_SCENARIOS="Hist_1970 Hist_2000 NoSAs_2000 NoCAs_2000 NoSAm_2000 NoAfME_2000 NoSEAs_2000 NoEAs_2000 NoNAm_2000 NoANZ_2000 NoEur_2000 NoSAs_1970 NoCAs_1970 NoSAm_1970 NoAfME_1970 NoSEAs_1970 NoEAs_1970 NoNAm_1970 NoANZ_1970 NoEur_1970 OnlySAs_2000 OnlyCAs_2000 OnlySAm_2000 OnlyAfME_2000 OnlySEAs_2000 OnlyEAs_2000 OnlyNAm_2000 OnlyANZ_2000 OnlyEur_2000 NoShip_2000 NoShip_1970 OnlyShip_2000"

    P16A_F_SCENARIOS="Hist_1970 Hist_2000 Zero_1970 Zero_2000"

    P16A_F_SCENARIOS="NoSAs_2000 NoCAs_2000 NoSAm_2000 NoAfME_2000 NoSEAs_2000 NoEAs_2000 NoNAm_2000 NoANZ_2000 NoEur_2000 NoShip_2000"

    P16A_F_SCENARIOS="OnlySAs_2000 OnlyCAs_2000 OnlySAm_2000 OnlyAfME_2000 OnlySEAs_2000 OnlyEAs_2000 OnlyNAm_2000 OnlyANZ_2000 OnlyEur_2000 OnlyShip_2000"

    P16A_F_SCENARIOS="NoSAs_1970 NoCAs_1970 NoSAm_1970 NoAfME_1970 NoSEAs_1970 NoEAs_1970 NoNAm_1970 NoANZ_1970 NoEur_1970 NoShip_1970"

Create cases by looping over scenarios, using FC5 as the base configuration.

    cd ~/beta/cesm1_2_2/scripts
    for scenario in $P16A_F_SCENARIOS
    do
    echo ${scenario}
    ./create_newcase -case ~/cesm_cases/p16a_F_${scenario} -res f19_g16 -compset FC5 -mach nsccbeta -pes_file pes/pes240.xml
    done

Loop through scenarios to configure, update user_nl_cam, build and submit.

    for scenario in $P16A_F_SCENARIOS
    do
    echo ${scenario}
    cd ~/cesm_cases/p16a_F_${scenario}
    ./cesm_setup
    rm -f user_nl_cam
    cp ~/scratch/00Data4Modeling/inputdataBSG/p16a_F/user_nl_cam_p16a_F_${scenario} user_nl_cam
    ./xmlchange STOP_OPTION=nyears
    ./xmlchange STOP_N=16
    ./xmlchange RESUBMIT=1
    ./p16a_F_${scenario}.build
    ./p16a_F_${scenario}.submit
    ln -s /home/users/academy/smart/benjamin/scratch/cesm_scratch/p16a_F_${scenario}/run/ scratch_run
    done
    cd ..

